In [61]:
import os
import csv
import cv2
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from random import shuffle

from sklearn.model_selection import train_test_split

from keras.layers.advanced_activations import ELU

In [90]:
images = []
samples = []
with open('driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

for sample in samples:
    
    name = 'data/IMG/'+sample[0].split('/')[-1]
    center_image = mpimg.imread(name)
    images.append(center_image)
    #image_RGB = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)


#train_samples, validation_samples = train_test_split(samples, test_size=0.2)
#train_samples = train_samples
#validation_samples = validation_samples


%matplotlib inline

In [92]:
#plt.imshow(images[1])

In [30]:
def generator(samples, batch_size=100):
    num_samples = (len(samples)//batch_size) *batch_size
    X_batch = np.zeros((batch_size, 160, 320, 3), dtype=np.float32)
    y_batch = np.zeros((batch_size,), dtype=np.float32)
    shuffle(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(1, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            for batch_sample in batch_samples[1:]:
                name = 'data/IMG/'+batch_samples[0].split('/')[-1]
                center_image = mpimg.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
                X_batch = np.array(images)
                y_batch = np.array(angles)
            yield X_batch, y_batch

In [72]:
def augment_brightness_camera_images(image):
    '''
    :param image: Input image
    :return: output image with randomly adjusted brightness
    '''

    # convert to HSV so that its easy to adjust brightness
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    # randomly generate the brightness reduction factor
    # Add a constant so that it prevents the image from being completely dark
    random_bright = .25+np.random.uniform()

    # Apply the brightness reduction to the V channel
    image1[:,:,2] = image1[:,:,2]*random_bright

    # convert to RBG again
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

In [93]:
def my_generator(samples, batch_size=32):
    N = (len(samples)//batch_size)*batch_size  # make the number of samples in 'lines' a multiple of batch_size

    X_batch = np.zeros((batch_size, 160, 320, 3), dtype=np.float32)
    y_batch = np.zeros((batch_size,), dtype=np.float32)
    
    while True:
        for i in range(0, N, batch_size):
            batch_start = i
            batch_stop = i + batch_size
            batch_lines = samples[batch_start:batch_stop]

            for j, line in enumerate(batch_lines):
                image = 'data/IMG/'+line[0].split('/')[-1]
                image = mpimg.imread(image)
                # crop
                #image = image[50:135,:,:]
                #image = cv2.resize(image, (200, 66))
                #image_BGR = cv2.resize(image,(200, 66), interpolation = cv2.INTER_AREA)
                #image = cv2.cvtColor(image_BGR,cv2.COLOR_BGR2RGB)
                image = augment_brightness_camera_images(image)
                
                #image_RGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                steering = float(line[3])

                X_batch[j] = image
                y_batch[j] = steering
           
            yield (X_batch, y_batch)


In [94]:
# compile and train the model using the generator function
train_generator = my_generator(train_samples, batch_size=32)
validation_generator = my_generator(validation_samples, batch_size=32)

ch, row, col = 3, 160, 320  # Trimmed image format

model = Sequential()

model.add(Lambda(lambda x: (x / 127.5) - 1.0, input_shape=(160,320,3),output_shape=(160,320,3)))
#model.add(Flatten(input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5, input_shape=(160,320,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=1)

model.save('model1.h5')

Epoch 1/1
6400/6428 [============================>.] - ETA: 0s - loss: 0.5613

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6432/6428 [==============================] - 25s - loss: 0.5585 - val_loss: 0.0166


In [96]:
#NVIDIA

# compile and train the model using the generator function
train_generator = my_generator(train_samples, batch_size=32)
validation_generator = my_generator(validation_samples, batch_size=32)

ch, row, col = 3, 160, 320

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(row, col, ch)))
model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(ELU())
model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dropout(.2))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(1))

model.compile(loss = 'mse', optimizer= 'adam')

#model.fit(X_train, y_train, validation_split = 0.2, shuffle = True, nb_epoch = 1)

model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=1)

model.save('model1.h5')


Epoch 1/1
6400/6428 [============================>.] - ETA: 0s - loss: 1.3155

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


6432/6428 [==============================] - 23s - loss: 1.3094 - val_loss: 0.2031


In [63]:
from keras.models import Sequential, Model
from keras.layers.core import Dense, Flatten, Lambda, Dropout
from keras.layers.core import Lambda
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D

In [30]:
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
#model.add(Flatten(input_shape=(160,320,3)))
model.add(Convolution2D(6,5,5, input_shape=(160,320,3),activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(6,5,5, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))
          

In [31]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train,y_train, validation_split=0.2, shuffle=True,nb_epoch=1)
model.save('model.h5')

Train on 19286 samples, validate on 4822 samples
Epoch 1/1
19286/19286 [==============================] - 2849s - loss: 0.9748 - val_loss: 0.0123
